In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

### Model Development Using StatsModels APIs

In [2]:
# Understanding the use of bias in linear models implemented in statsmodels.api
duncan_prestige = sm.datasets.get_rdataset("Duncan", "carData")
Y = duncan_prestige.data['income']
X = duncan_prestige.data['education']

In [3]:
duncan_prestige.data.head()

,type,income,education,prestige
accountant,prof,62,86,82
pilot,prof,72,76,83
architect,prof,75,92,90
author,prof,55,90,76
chemist,prof,64,86,90


In [4]:
duncan_prestige.data.to_csv('Duncan_Data_Set.csv')

In [5]:
X = sm.add_constant(X)
X.head()

/home/aumaron/Desktop/other_projects/venv/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


,const,education
accountant,1.0,86
pilot,1.0,76
architect,1.0,92
author,1.0,90
chemist,1.0,86


In [6]:
# hasconst=None
model = sm.OLS(Y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 income   R-squared:                       0.525
Model:                            OLS   Adj. R-squared:                  0.514
Method:                 Least Squares   F-statistic:                     47.51
Date:                Mon, 03 May 2021   Prob (F-statistic):           1.84e-08
Time:                        14:41:16   Log-Likelihood:                -190.42
No. Observations:                  45   AIC:                             384.8
Df Residuals:                      43   BIC:                             388.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         10.6035      5.198      2.040      0.048       0.120      21.087
education      0.5949      0.086      6.893      0.000       0.421       0.769
==============================================================================
Omnibus:                        9.841   Durbin-Watson:                   1.736
Prob(Omnibus):                  0.007   Jarque-Bera (JB):               10.609
Skew:                           0.776   Prob(JB):                      0.00497
Kurtosis:                       4.802   Cond. No.                         123.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
# hasconst=True (same result as that obtained using hasconst=None)
# model_true = sm.OLS(Y, X, hasconst=True)
# results_true = model.fit()
# results_true.summary()

In [8]:
# hasconst=False (same result as that obtained using hasconst=None)
# model_false = sm.OLS(Y, X, hasconst=False)
# results_false = model.fit()
# results_false.summary()

In [7]:
# Regularized regression
model = sm.OLS(Y,X)
temp = model.fit() #Step needed to populate 'normalized_cov_params'

In [8]:
model.normalized_cov_params

array([[ 9.30974511e-02, -1.34857729e-03],
       [-1.34857729e-03,  2.56600331e-05]])

In [9]:
# hasconst=None, regularized regression (lasso)
results_lasso = model.fit_regularized(method='elastic_net', alpha=1, L1_wt=1.0)
results_lasso.params

const        6.474853
education    0.654390
dtype: float64

In [10]:
# Step needed to generate output in the same format as that for OLS
results_lasso_format = sm.regression.linear_model.OLSResults(model, 
                                               results_lasso.params,        #Value populated using regularized regression
                                               model.normalized_cov_params) #Value populated using OLS
results_lasso_format.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 income   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.507
Method:                 Least Squares   F-statistic:                     46.20
Date:                Mon, 03 May 2021   Prob (F-statistic):           2.53e-08
Time:                        14:41:56   Log-Likelihood:                -190.75
No. Observations:                  45   AIC:                             385.5
Df Residuals:                      43   BIC:                             389.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.4749      5.236      1.237      0.223      -4.085      17.035
education      0.6544      0.087      7.528      0.000       0.479       0.830
==============================================================================
Omnibus:                       11.455   Durbin-Watson:                   1.726
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               14.359
Skew:                           0.810   Prob(JB):                     0.000762
Kurtosis:                       5.244   Cond. No.                         123.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Compare the results obtained using sklearn

In [11]:
# Linear regression with no penalty
lm_sk = linear_model.LinearRegression(fit_intercept=False) #X already includes the intercept column
lm_sk.fit(X, Y)
print(lm_sk.coef_)
print(lm_sk.intercept_)

[10.60349832  0.59485944]
0.0


In [12]:
#Linear regression with lasso penalty
lm_lasso_sk = linear_model.Lasso(alpha=1, fit_intercept=False)
lm_lasso_sk.fit(X, Y)
print(lm_lasso_sk.coef_)
print(lm_lasso_sk.intercept_)

[6.47667182 0.65436359]
0.0


### Compare the LASSO results for different values of alpha

In [13]:
model = sm.OLS(Y, X)
temp = model.fit() #Step needed to populate 'normalized_cov_params'

In [14]:
current_val_alpha = 1.5

# Lasso using statsmodels
results_lasso = model.fit_regularized(method='elastic_net', alpha=current_val_alpha, L1_wt=1.0)
print(results_lasso.params)

# Lasso using sklearn
lm_lasso_sk = linear_model.Lasso(alpha=current_val_alpha, fit_intercept=False)
lm_lasso_sk.fit(X, Y)
print(lm_lasso_sk.coef_)

const        4.410506
education    0.684156
dtype: float64
[4.41158416 0.68413992]


In [15]:
current_val_alpha = 2.5

# Lasso using statsmodels
results_lasso = model.fit_regularized(method='elastic_net', alpha=current_val_alpha, L1_wt=1.0)
print(results_lasso.params)

# Lasso using sklearn
lm_lasso_sk = linear_model.Lasso(alpha=current_val_alpha, fit_intercept=False)
lm_lasso_sk.fit(X, Y)
print(lm_lasso_sk.coef_)

const        0.281811
education    0.743687
dtype: float64
[0.28193285 0.74368498]


In [16]:
current_val_alpha = 5

# Lasso using statsmodels
results_lasso = model.fit_regularized(method='elastic_net', alpha=current_val_alpha, L1_wt=1.0)
print(results_lasso.params)

# Lasso using sklearn
lm_lasso_sk = linear_model.Lasso(alpha=current_val_alpha, fit_intercept=False)
lm_lasso_sk.fit(X, Y)
print(lm_lasso_sk.coef_)

const        0.00000
education    0.74708
dtype: float64
[0.        0.7470799]


### Predicted Values

In [17]:
results_lasso.predict(X.iloc[0:2, :])

accountant    64.248871
pilot         56.778072
dtype: float64

In [18]:
# Obtain 95% confidence interval for the predicted values
# Only feasible for non-regularized models
print(temp.predict(X.iloc[0:2, :]))
prstd, iv_l, iv_u = wls_prediction_std(temp, X.iloc[0:2, :])

accountant    61.761410
pilot         55.812816
dtype: float64


In [19]:
iv_l

array([26.53931647, 20.83620125])

In [20]:
iv_u

array([96.98350385, 90.78943027])

In [21]:
conf_int = pd.DataFrame(data=np.hstack([iv_l[:, None], iv_u[:, None]])
                        , columns=['Lower', 'Upper'])
conf_int

,Lower,Upper
0,26.539316,96.983504
1,20.836201,90.789430


### Compare the result with that obtained using glmnet in R

In [22]:
X_new = duncan_prestige.data[['education', 'prestige']]
print(X_new.head())

X_new = sm.add_constant(X_new)
X_new.head()

            education  prestige
accountant         86        82
pilot              76        83
architect          92        90
author             90        76
chemist            86        90


/home/aumaron/Desktop/other_projects/venv/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


,const,education,prestige
accountant,1.0,86,82
pilot,1.0,76,83
architect,1.0,92,90
author,1.0,90,76
chemist,1.0,86,90


In [23]:
# Initial parameter estimates for lasso are NOT obtained using OLS
model_r = sm.OLS(Y, X_new)
current_val_alpha = 1
results_lasso = model_r.fit_regularized(method='elastic_net', alpha=current_val_alpha, L1_wt=1.0)
print(results_lasso.params)

const        6.068219
education    0.102869
prestige     0.615934
dtype: float64


In [24]:
# Initial parameter estimates for lasso are obtained using OLS
model_r = sm.OLS(Y, X_new)
temp = model_r.fit() 

current_val_alpha = 1
results_lasso = model_r.fit_regularized(method='elastic_net', alpha=current_val_alpha, L1_wt=1.0
                                        , start_params=temp.params)
print(results_lasso.params)

const        6.297060
education    0.091185
prestige     0.624407
dtype: float64


In [25]:
# alpha=0 gives the same result as that obtained in R
model_r = sm.OLS(Y, X_new)
temp = model_r.fit() #Step needed to populate 'normalized_cov_params'

results_lasso = model_r.fit_regularized(method='elastic_net', alpha=0, L1_wt=1.0
                                        , start_params=temp.params)
print(results_lasso.params)

const        10.426361
education     0.032263
prestige      0.623724
dtype: float64
